In [1]:
import pyomo.environ as pyo
import numpy as np

In [15]:
data = []
N = 443
with open(r'data.txt', 'r') as f:
    file_lines = f.readlines()
for line in file_lines:
    line =line.replace("\n", "")
    line = line.split(' ')
    for i in line:
        try:
            data.append(int(i))
        except:
            continue
data= np.array(data).reshape((N,N))

In [16]:
data

array([[ 0, 11, 10, ..., 14, 24, 24],
       [ 0,  0, 11, ..., 15, 24, 24],
       [11, 12,  0, ..., 15, 25, 25],
       ...,
       [23, 21, 23, ...,  0, 12, 12],
       [12, 14, 11, ..., 18,  0, 26],
       [23, 21, 23, ..., 16, 14,  0]])

In [4]:
model = pyo.ConcreteModel()

## Sets

In [5]:
model.D = pyo.Set(initialize=range(1,N))
model.N = pyo.Set(initialize=range(N)) 

model.A = pyo.Set(initialize=[(i, j) for i in model.N for j in model.N if i != j]) #set of arcs

## Variables

In [6]:
model.X = pyo.Var(model.A, within=pyo.Binary) # decision to move along arc ij
model.T = pyo.Var(model.N, within=pyo.NonNegativeIntegers, bounds = (0,N))

## Constraints

In [7]:
M =1e8

In [8]:
def arcs_out_rule(model, i): 
    return sum(model.X[i, j] for j in model.N if i != j) == 1
model.arcs_out_rule = pyo.Constraint(model.N, rule=arcs_out_rule)

def arcs_in_rule(model, i): 
    return sum(model.X[j, i] for j in model.N if i != j) == 1
model.arcs_in_rule = pyo.Constraint(model.N, rule=arcs_in_rule)

def subtour_elmination(model,i,j):
    if i == j:
        return pyo.Constraint.Skip
    else:
        return model.T[j] >= model.T[i] +1 - M*(1-model.X[i,j])
model.subtour_elmination = pyo.Constraint(model.D, model.D, rule=subtour_elmination)

## Objective

In [9]:
#obj to minimise travel cost
model.obj = pyo.Objective(
    expr=sum(
        model.X[i, j] * data[i,j]
        for (i, j) in model.A), 
    sense=pyo.minimize,
)

In [10]:
TIME_LIMIT = 180
THREADS = 4

SOVLER_ENGINE = 'gurobi'
#solvers glpk  appsi_highs cplex gurobi gurobi_persistent


solver = pyo.SolverFactory(SOVLER_ENGINE)
#solver.options['tmlim'] = TIME_LIMIT

if SOVLER_ENGINE == 'cbc':
        solver.options['seconds'] = TIME_LIMIT
elif SOVLER_ENGINE == 'glpk':
        solver.options['tmlim'] = TIME_LIMIT
elif SOVLER_ENGINE == 'appsi_highs':
        solver.options['time_limit'] = TIME_LIMIT
        solver.options['parallel'] = 'on'
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'cplex':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'gurobi':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS

elif SOVLER_ENGINE == 'gurobi_persistent':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
        solver.set_instance(model) # remove model from  solver.solve(tee=True)
        solver.set_gurobi_param("PoolSolutions", 500)
        solver.set_gurobi_param("PoolSearchMode", 0)

#solver.options['time_limit'] = TIME_LIMIT
#
# sol = solver.solve(tee= True)
sol = solver.solve(model,tee= True) #, warmstart=True , logfile= 'log.txt'

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-13
Read LP format model from file C:\Users\caleb\AppData\Local\Temp\tmpo96qy50q.pyomo.lp
Reading time = 1.12 seconds
x1: 195808 rows, 196248 columns, 976378 nonzeros
Set parameter TimeLimit to value 180
Set parameter Threads to value 4
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 4 threads

Optimize a model with 195808 rows, 196248 columns and 976378 nonzeros
Model fingerprint: 0x026da6d9
Variable types: 0 continuous, 196248 integer (195806 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 4e+02]
  RHS range        [1e+00, 1e+08]
Presolve time: 2.46s
Presolved: 195808 rows, 196248 columns, 976378 nonzeros
Variable types: 0 continuous, 19

In [11]:
print (model.obj())
print ('Total travel time:', sum(model.X[i,j].value * data[i,j]
                                               for (i,j) in model.A))

2720.0
Total travel time: 2720.0


In [12]:
node = 0
route = []
for i in model.N:
    for j in model.N:
        if node == j:
            continue
        elif np.isclose(model.X[node, j].value, 1, atol=1e-1):
            route.append(node)
            node = j
            break
    if node == 0:
        route.append(node)
        break

In [13]:
np.array(route)

array([  0,  10, 405, 189,  86,  32, 249, 237, 401, 380,  93,  96, 308,
       367, 354, 102, 289, 340, 275,  22, 278, 316, 298, 250, 103, 120,
       225, 305, 264, 255, 180,  64, 408, 186, 183, 150,  76, 286, 432,
       409,  47, 143, 216,  95, 134, 307,  15, 290, 260, 136, 246, 214,
       100, 166, 413,  58, 384, 388, 386,  81, 178, 361, 416, 280,  98,
       147,  89, 196, 212, 224,  73, 221,  13, 364, 292, 179, 126, 199,
        80, 393, 391, 110, 259, 121, 343, 244, 232,   7,  66,  44,   9,
       149, 148, 132, 251, 376, 313, 374, 368, 412, 203,  79, 210, 118,
       213, 294,   4,   1, 125,  52, 399, 320, 142,  97, 122, 222, 141,
       397, 355, 115,  34, 288, 226, 403, 365, 359, 197,  92,  74, 228,
       268, 234, 327, 310,  17, 124, 239,   8,  83, 315, 346, 318, 271,
       209, 116, 211,  48, 415, 193, 379, 363, 205, 395, 431,  87,  28,
       356,  40, 245, 366, 282, 106, 167,  61,  16, 235, 101, 284, 247,
       202, 176, 394,  20, 430, 418, 358, 190, 104, 333, 339, 42

In [14]:
len(route)

444